In [1]:
#%pip install requests python-dotenv pypdf langchain langchain-openai langchain-community langchain-core langchain-experimental langgraph tavily-python ipython faiss-cpu

In [1]:
import os
from dotenv import load_dotenv


# Load environment variables from .env file
load_dotenv()

True

## Parsing and Chunking User's Tax Return (Skip for Demo)

In [2]:
pdf = "../data/sample_returns/dummy1.pdf"
# question = "How do I improve my tax return?"

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(pdf)
documents = loader.load()

print(documents[0].page_content)

GOLDEN STATE ACCOUNTING INC.
1221 BRIDGEWAY SUITE 2
SAUSALITO, CA 94965
415-331-9900
May 31, 2024
Joseph W and Stacy T Smith
16023 Via Del Alba
Rancho Santa Fe, CA 92067
Dear Joe and Stacy, 
Your 2023 Federal Individual Income Tax return will be
electronically filed with the Internal Revenue Service upon receipt
of a signed Form 8879 - IRS e-file Signature Authorization.  There
is a balance due of $700.  
Make your check payable to the "United States Treasury" and mail
your Form 1040-V payment voucher on or before April 15, 2024 to: 
INTERNAL REVENUE SERVICE
P.O. BOX 802501
CINCINNATI, OH 45280-2501
The deductible contribution to your spouse's Health Savings Account
for 2023 is $5,350.  To ensure that your spouse's contribution is
allowable, $5,350 must be deposited to your spouse's account on or
before April 15, 2024.  
Your 2023 California Individual Income Tax Return will be
electronically filed with the Franchise Tax Board upon receipt of a
signed Form 8879 - California e-file Sign

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

loader = PyPDFLoader(pdf)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

chunks = splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(chunks, embeddings)
vectorstore.save_local("../data/faiss_index")
vectorstore = FAISS.load_local("../data/faiss_index", embeddings)

## Defining the AgentState

In [6]:
from typing import TypedDict, List, Dict, Any

class AgentState(TypedDict):
    # TODO - if no question is given, could use 'How do I improve this tax return?'
    users_question: str

    # TODO - Maybe replace users question by just using last element
    conversation_history: List[Dict[str, str]]

    sub_tasks: List[str]
    current_subtask: str

    # List of everything retrieved so far from tools
    retrieval_history: List[Dict[str, Any]]

    sub_task_solutions: List[str]

    # Used for Subtask_Router to determine next node to go to.
    next_node: str

    # chain_of_thought should only be used for debugging!
    chain_of_thought: str
    final_answer: str

#### Choosing Agent Model

In [7]:
from langchain_openai import ChatOpenAI

openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key=openai_api_key)

## Tools to be Used in the Nodes

In [8]:
from langchain_core.tools import tool

#### Python REPL Tool (Allows an LLM to make its own calculations)

In [9]:
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()

In [10]:
@tool
def calculator(code: str) -> str:
    """Execute a Python expression or snippet and return the printed result."""
    return python_repl.run(code)

#### Semantic Document Search Tool

In [11]:
@tool
def tax_return_doc_search(query: str) -> str:
    """Search the embedded tax document for relevant information."""
    results = vectorstore.similarity_search(query, k=3)
    return "\n\n".join([doc.page_content for doc in results])

#### TavilySearch (Web Search)

In [12]:
from tavily import TavilyClient

tavily_api_key = os.getenv("TAVILY_API_KEY")
tavily_client = TavilyClient(tavily_api_key)

In [13]:
@tool
def web_search(query: str) -> str:
    """Perform a web search and return the results."""
    try:
        query = query.replace('"', '')
        response = tavily_client.search(query=query, max_results=3)
        hits = response.get("results", [])
        if not hits:
            return "No relevant web results found for that query."
        return "\n\n".join(f"{r.get('title', '')}\n{r.get('url', '')}\n{r.get('content', '')}" for r in hits)
    except Exception as e:
        return f"Web search failed due to: {e}"

In [14]:
tools = [calculator, tax_return_doc_search, web_search]

## Defining the Nodes

#### Helper Methods

In [15]:
def parse_subtasks(text: str) -> List[str]:
    """
    Parses out bullet points or line-separated items from LLM output.
    Example approach: split on newlines, filter out empty lines.
    """
    lines = [line.strip("-• ").strip() for line in text.split("\n") if line.strip()]
    return [line for line in lines if line]


def parse_tool_decision(researcher_response: str) -> (str, str):
    """
    Parses the tool decision response from the LLM and returns the chosen tool and input.
    :param researcher_response: The LLM's decision response to what tool it should use
    :return: chosen_tool, tool_input
    """
    chosen_tool = None
    tool_input = None
    for line in researcher_response.split("\n"):
        line_lower = line.lower()
        if line_lower.startswith("tool:"):
            chosen_tool = line.split(":", 1)[1].strip()
        elif line_lower.startswith("toolinput:"):
            tool_input = line.split(":", 1)[1].strip()

    if chosen_tool is None:
        chosen_tool = "web_search"
    if tool_input is None:
        # tool_input = state['current_subtask']
        tool_input = researcher_response

    return chosen_tool, tool_input


def invoke_tool(tool_name: str, tool_input: str) -> str:
    """
    Invokes the specified tool with the given input.
    """
    for t in tools:
        if t.name == tool_name:
            result = t.invoke(tool_input)
            return format_result(result)

    raise ValueError(f"Tool '{tool_name}' not found.")


def format_result(data) -> str:
    """
    Safely convert the tools' output (string, list, dict, or other)
    to a single string. If it's a list, we'll join its elements
    line-by-line. If it's a dict or any other type, we'll just
    string-ify it.
    """
    if isinstance(data, str):
        return data

    if isinstance(data, list):
        lines = []
        for item in data:
            if isinstance(item, (list, dict)):
                # nested structure? string-ify
                lines.append(str(item))
            else:
                lines.append(str(item))  # or item if it's already str
        return "\n".join(lines)

    # If it's a dict, or any other type, just do str(...)
    return str(data)

In [16]:
number_of_tasks = 7

def planner(state: AgentState) -> AgentState:
    """
    Decomposes the user's main question into actionable sub_tasks using the LLM.
    """

    state["chain_of_thought"] += f"Planner Node: deriving sub_tasks based on '{state["users_question"]}'...\n"

    planner_prompt = f"""You are a planning assistant.
    The user's main question is: "{state['users_question']}".
    Conversation so far (if relevant):
    {state["conversation_history"]}

    Please break the user's question into around {number_of_tasks} actionable sub-tasks or key points we should investigate.
    Each sub-task should be concise, focusing on a specific aspect of the question or problem.
    Only list the sub-tasks, with no extra explanation.
    """

    try:
        plan_response = llm.invoke(planner_prompt).content
    except Exception as e:
        plan_response = f"Error during planning: {e}"
        state["chain_of_thought"] += f"\nPlanner Error: {e}\n"

    generated_subtasks = parse_subtasks(plan_response)
    state["sub_tasks"] = generated_subtasks

    if generated_subtasks:
        state["current_subtask"] = generated_subtasks[0]

    state["chain_of_thought"] += f"\nPlanner Node output:\n{plan_response}\n"

    return state


def researcher(state: AgentState) -> AgentState:
    """
    Asks the LLM which tool to use for the current_subtask.
    Then calls that tool (calculator, source_doc_search, or web_search).
    Appends the result to retrieval_history.
    """

    current_subtask = state["current_subtask"]
    if not current_subtask:
        return state

    state["chain_of_thought"] += f"\nResearcher: Deciding tool for '{current_subtask}'\n"


    # Incorporate conversation or sub_task_solutions so far:
    #Should access content from state["retrieval_history"] where subtask matches
    solutions_text = "\n".join(state["sub_task_solutions"])
    conversation_text = "\n".join(
        f"{m['role'].upper()}: {m['content']}"
        for m in state["conversation_history"]
    )

    researcher_prompt = f"""You are a researcher that decides which tool to use.
    Current subtask: "{current_subtask}"
    Context from sub_task_solutions:
    {solutions_text}

    Tools available:
    1) calculator
    2) tax_return_doc_search
    3) web_search

    Which tool should we use next to gather info for this subtask?
    Also provide the input we should pass to that tool.

    Output format:
    Tool: <one of calculator/source_doc_search/web_search>
    ToolInput: <string input for that tool>
    """

    try:
        researcher_response = llm.invoke(researcher_prompt).content
    except Exception as e:
        researcher_response = f"Error during research: {e}"
        state["chain_of_thought"] += f"\nResearcher Error: {e}\n"

    state["chain_of_thought"] += f"\nResearcher response:\n{researcher_response}\n"

    chosen_tool, tool_input = parse_tool_decision(researcher_response)

    content = invoke_tool(chosen_tool, tool_input)

    entry = {
        "subtask": current_subtask,
        "tool_used": chosen_tool,
        "tool_input": tool_input,
        "content": content
    }
    state["retrieval_history"].append(entry)

    state["chain_of_thought"] += f"\nTool output for '{current_subtask}':\n{content}\n"

    return state


def analyzer(state: AgentState) -> AgentState:
    """
    Looks at all retrieval data for the current_subtask
    Creates or updates a solution for this subtask.
    """

    current_subtask = state["current_subtask"]
    if not current_subtask:
        return state

    relevant_entries = [
        rh for rh in state["retrieval_history"]
        if rh["subtask"] == current_subtask
    ]
    subtask_content = "\n".join(e["content"] for e in relevant_entries)

    analyzer_prompt = f"""You are an analyzer.
    The current subtask is: "{current_subtask}".
    Here is the information we retrieved:
    {subtask_content}

    Please create or update a partial solution for this subtask that integrates the new findings.
    Your answer should be thorough and include any relevant details previously found
    """
    try:
        analyzer_response = llm.invoke(analyzer_prompt).content
    except Exception as e:
        analyzer_response = f"Error during analysis: {e}"
        state["chain_of_thought"] += f"\nAnalyzer Error: {e}\n"

    state["sub_task_solutions"].append(analyzer_response)

    # state["chain_of_thought"] += f"Analyzer Node: All entries for this subtask so far:\n{relevant_entries}\n"
    state["chain_of_thought"] += f"\nAnalyzer Node solution for '{current_subtask}':\n{analyzer_response}\n"

    return state


def subtask_router(state: AgentState) -> AgentState:
    """
    The LLM decides:
    - If we need more research for the current subtask, go back to 'researcher' node
    - If we are done with this subtask, move on to next subtask and go to 'researcher' node
      or proceed to 'synthesizer' if no more subtasks remain.

    Returns the name of the next node: 'researcher' or 'synthesizer'
    """
    sub_tasks = state["sub_tasks"]
    current_subtask = state["current_subtask"]
    if not sub_tasks or current_subtask not in sub_tasks:
        state["chain_of_thought"] += "\nSubtask Router: no valid subtask. Going to synthesizer.\n"
        state["next_node"] = "synthesizer"
        return state

    # Let the LLM decide if we need more research
    solutions_text = "\n".join(state["sub_task_solutions"][-3:])

    router_prompt = f"""You are a subtask router that decides whether
    we have fully researched a subtask.
    The user's main question is: "{state['users_question']}"
    We have solutions for this subtask: "{current_subtask}"
    Here are recent partial solutions this subtask:
    {solutions_text}

    Do we have enough information to finalize this subtask, or do we need more research?
    Answer 'MORE' if we should research more, or 'DONE' if we can move on.
    We should research more if the information found is not relevant
    to the subtask and not relevant the user's main question.
    """

    router_response = llm.invoke(router_prompt).content.lower()
    state["chain_of_thought"] += f"\nSubtask Router Node: Router's decision: {router_response}\n"

    if "more" in router_response:
        # Continue researching this subtask
        state["chain_of_thought"] += "Subtask Router Node: Continuing research for this subtask.\n"
        state["next_node"] = "researcher"
        return state
    else:
        # Move on to next subtask or final
        index = sub_tasks.index(current_subtask) + 1
        if index < len(sub_tasks):
            new_subtask = sub_tasks[index]
            state["current_subtask"] = new_subtask
            state["chain_of_thought"] += f"Subtask Router Node: Next subtask '{new_subtask}'.\n"
            state["next_node"] = "researcher"
            return state
        else:
            state["chain_of_thought"] += f"Subtask Router Node: All subtasks done. Proceed to synthesizer.\n"
            state["next_node"] = "synthesizer"
            return state


def synthesizer(state: AgentState) -> AgentState:
    """
    Gathers partial solutions from all subtasks and composes a final answer.
    """
    question = state["users_question"]
    solutions_text = "\n\n".join(state["sub_task_solutions"])

    synth_prompt = f"""You are the final synthesizer.
    User's question: "{question}"

    Below are the solutions for each subtask:

    {solutions_text}

    Please synthesize these findings into a final, coherent response
    that thoroughly addresses the user's main question. Structure it
    with an introduction, a well-organized body, and a conclusion.
    """
    try:
        final_response = llm.invoke(synth_prompt)
        final_content = final_response.content
    except Exception as e:
        final_content = f"Error during synthesis: {e}"
        state["chain_of_thought"] += f"\nSynthesizer Error: {e}\n"

    state["final_answer"] = final_content

    state["chain_of_thought"] += "\nSynthesizer Node: Produced final answer.\n"

    return state


## Additional tools

In [17]:
def call_llm(prompt: str) -> str:
    """
    Centralized helper for invoking the LLM.
    Wraps llm.invoke() in a try/except block and returns the LLM's content response.
    """
    try:
        response = llm.invoke(prompt)
        return response.content
    except Exception as e:
        # Log the error in the chain-of-thought for debugging.
        return f"Error invoking LLM: {e}"


def validator(state: AgentState) -> AgentState:
    """
    Validator node:
    Reviews the current subtask's partial solution for consistency and possible gaps.
    It aggregates retrieved information related to the current subtask and asks the LLM to validate it.
    Sets a new state variable 'validator_feedback' that can be examined by subsequent nodes.
    """
    current_subtask = state.get("current_subtask", "")
    if not current_subtask:
        return state

    # Gather related information from retrieval_history for the current subtask.
    relevant_entries = [
        entry for entry in state.get("retrieval_history", [])
        if entry.get("subtask") == current_subtask
    ]
    aggregated_content = "\n".join([entry.get("content", "") for entry in relevant_entries])

    # Create a prompt to ask the LLM for validation.
    validator_prompt = (
        f"You are a validator node. The current subtask is: '{current_subtask}'.\n"
        f"Below is the aggregated partial solution information:\n{aggregated_content}\n\n"
        "Please evaluate whether the solution has any inconsistencies or gaps. "
        "If everything looks good, respond with 'OK'. Otherwise, describe what should be improved."
    )

    validator_response = call_llm(validator_prompt)
    state["validator_feedback"] = validator_response
    state["chain_of_thought"] += f"\nValidator output for '{current_subtask}': {validator_response}\n"

    return state


def context_enhancer(state: AgentState) -> AgentState:
    """
    Context Enhancer node:
    Aggregates the conversation history and the most recent subtask solutions to build an enhanced
    context string. This context can later be used to improve the clarity and accuracy of LLM prompts.
    """
    # Aggregate conversation history (if available).
    conversation_text = "\n".join(
        f"{msg.get('role', 'UNKNOWN')}: {msg.get('content', '')}"
        for msg in state.get("conversation_history", [])
    )

    # Get the last three partial solutions (if available).
    recent_solutions = state.get("sub_task_solutions", [])
    latest_solutions = "\n".join(recent_solutions[-3:]) if recent_solutions else ""

    enhanced_context = f"Conversation History:\n{conversation_text}\n\nRecent Solutions:\n{latest_solutions}"
    state["enhanced_context"] = enhanced_context
    state["chain_of_thought"] += "\nContext Enhancer Node: Enhanced context built for subsequent nodes.\n"

    return state


def fallback_tool(state: AgentState, failed_tool: str, tool_input: str) -> str:
    """
    Fallback tool method:
    If a primary tool invocation fails (or returns unsatisfactory results), this method provides a fallback.
    For example, if 'tax_return_doc_search' fails, we might default to 'web_search'.
    It logs the fallback action to the chain_of_thought.
    """
    state["chain_of_thought"] += (
        f"\nFallback: Detected failure from primary tool '{failed_tool}' with input '{tool_input}'. "
        "Attempting fallback with 'web_search'.\n"
    )
    fallback_result = invoke_tool("web_search", tool_input)
    state["chain_of_thought"] += f"\nFallback tool result: {fallback_result}\n"
    return fallback_result




## Confidence Scoring

In [18]:
def confidence_scoring(state: AgentState) -> AgentState:
    """
    Confidence Scoring node:
    Analyzes the current subtask's latest partial solution to assign a confidence score.
    The LLM is asked to rate the solution on a scale of 1-10 and provide brief reasoning.
    This score is stored in the state under 'confidence_score'.
    """
    current_subtask = state.get("current_subtask", "")
    if not current_subtask:
        return state

    # Retrieve the latest partial solution for the current subtask.
    if state.get("sub_task_solutions"):
        latest_solution = state["sub_task_solutions"][-1]
    else:
        latest_solution = "No solution found."

    score_prompt = (
        f"You are a confidence scoring assistant. For the subtask '{current_subtask}', please evaluate the "
        f"following solution on a scale from 1 (low confidence) to 10 (high confidence), and provide a brief "
        f"justification for the score. Here is the solution:\n{latest_solution}\n\n"
        "Provide your answer in the format: Score: <number>, Explanation: <brief explanation>."
    )
    
    score_response = call_llm(score_prompt)
    state["confidence_score"] = score_response
    state["chain_of_thought"] += f"\nConfidence scoring for '{current_subtask}': {score_response}\n"
    
    return state

## Plotting the Nodes

In [19]:
from langgraph.graph import StateGraph, START, END

graph = StateGraph(AgentState)

graph.add_node("planner", planner)
graph.add_node("researcher", researcher)
graph.add_node("analyzer", analyzer)
graph.add_node("subtask_router", subtask_router)
graph.add_node("synthesizer", synthesizer)

In [20]:
graph.add_edge(START, "planner")
graph.add_edge("planner", "researcher")
graph.add_edge("researcher", "analyzer")
graph.add_edge("analyzer", "subtask_router")

graph.add_conditional_edges("subtask_router", lambda state: state["next_node"], {
    "researcher": "researcher",
    "synthesizer": "synthesizer",
})

graph.add_edge("synthesizer", END)

In [21]:
# Compile the graph into an agent application
agent_app = graph.compile()

In [22]:
from IPython.display import Image, display

try:
    display(Image(agent_app.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    print("Error displaying the graph.")
    pass

Error displaying the graph.


## Running the Agent

In [23]:
# pdf = "../data/sample_returns/dummy1.pdf"
question="What are steps that I can take to beat Minecraft as quick as possible?"

In [24]:
initial_state = AgentState(
    users_question=question,
    conversation_history=[],
    sub_tasks=[],
    current_subtask="",
    retrieval_history=[],
    sub_task_solutions=[],
    next_node="",
    chain_of_thought="",
    final_answer=""
)

# Invoke the agent with the initial state
result_state = agent_app.invoke(initial_state)

# Print or use the final answer from the state
print("CHAIN OF THOUGHT (debug):\n", result_state["chain_of_thought"])
print(result_state["final_answer"])

CHAIN OF THOUGHT (debug):
 Planner Node: deriving sub_tasks based on 'What are steps that I can take to beat Minecraft as quick as possible?'...

Planner Node output:
1. Gather essential resources quickly (wood, stone, iron).
2. Craft necessary tools and weapons (pickaxe, sword, armor).
3. Locate and explore a village for supplies and trading.
4. Mine for diamonds and craft diamond gear.
5. Enter the Nether to collect blaze rods and ender pearls.
6. Craft Eyes of Ender and locate the Stronghold.
7. Defeat the Ender Dragon in the End dimension.

Researcher: Deciding tool for '1. Gather essential resources quickly (wood, stone, iron).'

Researcher response:
To gather essential resources like wood, stone, and iron, the most appropriate tool from the list provided would be a web search. This is because a web search can provide information on where and how to gather these resources quickly, including techniques, locations, and tools needed.

Tool: web_search  
ToolInput: "how to quickly gat

## Confidence Score for Result State

In [25]:
# Perform confidence scoring on the result state
result_state = confidence_scoring(result_state)

# Print the confidence score and explanation
print("Confidence Score and Explanation:")
print(result_state["confidence_score"])

Confidence Score and Explanation:
Score: 9, Explanation: The solution provides a comprehensive and well-rounded strategy for efficiently defeating the Ender Dragon in Minecraft. It covers essential preparation, including gear, potions, and tools, and offers a clear understanding of the dragon's mechanics and attack patterns. The combat strategy is detailed, emphasizing the importance of destroying End Crystals and managing Endermen. Additionally, the solution includes practical tips like using a Respawn Anchor and practicing in creative mode. The only minor improvement could be the inclusion of more advanced gear options, such as Netherite equipment, which would slightly enhance the strategy.


## Cleanup

In [26]:
# Delete all the users information in vectorstore

